In [ ]:
import shlex
from tqdm import tqdm
from IPython.display import display

from GTTA import config_summoner
from GTTA.kernel.main import initialize_model_dataset
from GTTA.utils.args import args_parser
from GTTA.utils.load_manager import load_atta_algorithm, load_alg
from GTTA.utils.load_manager import load_tta_algorithm
from GTTA.utils.logger import load_logger

argv = shlex.split('--task train --config_path TTA_configs/PACS/base.yaml --gpu_idx 2 --exp_round 1')
print(argv)
args = args_parser(argv)
config = config_summoner(args)
# load_logger(config)

alg = load_atta_algorithm(config)

from pathlib import Path
save_path = Path(config.ckpt_dir) / 'exp'
save_path.mkdir(parents=True, exist_ok=True)

In [2]:
import random

import numpy as np
import torch

def reset_random_seed(seed):
    # Fix Random seed
    random.seed(config.random_seed)
    np.random.seed(config.random_seed)
    torch.manual_seed(config.random_seed)
    torch.cuda.manual_seed(config.random_seed)
    torch.cuda.manual_seed_all(config.random_seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [3]:
from tqdm.notebook import tqdm
import time
for i in tqdm(range(100)):
    print(1)
    # time.sleep(1)

  0%|          | 0/100 [00:00<?, ?it/s]

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


$\mathcal{D}$: joint distribution - $P(X, Y)$

$\mathcal{X}$ input space. random variable $X$ on the $\mathcal{X}$ with probability $P(X)$.
$\hat{Y}\sim P_\phi(Y|X)$

$\mathcal{D}_{\phi, \mathcal{S}} = P_\phi(Y|X)P_\mathcal{S}(X)$

# Storyline

Test-time adaptation task can be considered as **multiple domain adaptation tasks**, where source domain expands while new target domain streaming in continuously. Initially, before the test phase begins, we have the initial training set $\mathcal{D}_S(0)$. At time $t = 1$, the training and test data are defined as $\mathcal{D}_S(1) = {\mathcal{D}_S(0)}$ and $\mathcal{D}_T(1)$. For every time point $t$, given that $\mathcal{U}_T(t_i) \subseteq \mathcal{D}_T(t_i)$, the training and test data can be expressed as
$$\mathcal{D}_S(t)=\{\mathcal{D}_S(0),\mathcal{U}_T(1),\mathcal{U}_T(2),\cdots,\mathcal{U}_T(t-1)\}\text{ and } \mathcal{D}_T(t).$$
The subset $\mathcal{U}_T(t-1)$ is actively labeled before each time period $t$ starts.

Therefore, at the time frame $t$, we are intrinsically considering a domain adaptation problem where the source domain is $\mathcal{D}_S(t)$ and the target domain is $\mathcal{D}_T(t)$.

For simplicity, we consider a domain adaptation problem where we have a source domain $D_\mathcal{S}$ and a target domain $D_\mathcal{T}$. The learning theory indicates that the empirical target domain performance (error) $\epsilon_T$ is mainly bounded by two parts:

* The learning deviation between empirical errors and true errors: $|\epsilon^S(h) - \hat{\epsilon}^S(h)|$ and $|\epsilon^T(h) - \hat{\epsilon}^T(h)|$. This term is denoted as $B$.
* The divergence between $D_\mathcal{S}$ and $D_\mathcal{T}$. We denote this term as $A \in [0, 1]$

## Experiment of empirical target error with different $\alpha$ and $\beta$
* $\alpha$: learning weights - $\mathcal{L} = (1 - \alpha) * \mathcal{L}_\mathcal{S} + \alpha * \mathcal{L}_\mathcal{T}$
* $\beta$: number of samples weights - $\mathcal{D} = \mathcal{D}_\mathcal{S}.sample((1 - \beta) * m) + \mathcal{D}_\mathcal{T}.sample(\beta * m)$. $m$ is the total sample number of the dataset $D$.

Given $\beta$, when the VC-dimension $d$ is much larger than $A$, we can prove that this bound is dominated by $B$. Because $B$ reaches its optimum when $\alpha=\beta$, we conduct the following experiments to illustrate the consistency between the theory and the practice.

In the following first experiment, we train models using $m=2000$ samples with different $\alpha$ and $\beta$, and test these models on the target domain. We draw the target domain loss surface as following. According to the alignment between the $\alpha=\beta$ line and the loss minimums, we can conclude that **1) the VC-dimension $d$ is indeed much larger than $A$, 2) the theoretical bound is practical.**

In [4]:
# Prepare for training
import numpy as np
D_S = alg.dataset[0]
D_T = alg.target_dataset

import torch
import torch.nn as nn
import copy
from tqdm import tqdm_notebook
from torch.utils.data import ConcatDataset
from GTTA.data.good_loaders.fast_data_loader import InfiniteDataLoader, FastDataLoader
from GTTA.utils.register import register
import pandas as pd

def train_new_model(alpha, beta, device, m=2000, tol = 1e-3, source_trained=False):
    S_choice = np.random.choice(len(D_S), int((1 - beta) * m), replace=False)
    T_choice = np.random.choice(len(D_T), int(beta * m), replace=False)
    D_S_loader = InfiniteDataLoader(D_S,
                                    weights=None,
                                    batch_size=config.train.train_bs,
                                    num_workers=config.num_workers,
                                    subset=S_choice)

    D_T_loader = InfiniteDataLoader(D_T,
                                    weights=None,
                                    batch_size=config.train.train_bs,
                                    num_workers=config.num_workers,
                                    subset=T_choice)

    if source_trained:
        model = copy.deepcopy(alg.model.cpu()).to(device)
    else:
        encoder = register.models[config.model.name](config)
        model = nn.Sequential(encoder,
                              nn.Linear(encoder.n_outputs, config.dataset.num_classes)).to(device)
    model.train()
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)
    lowest_loss = float('inf')
    stop_count = 0
    DT_loader = iter(zip(D_S_loader, D_T_loader))
    epoch_pbar = tqdm_notebook(range(100))
    for epoch in epoch_pbar:
        mean_loss = []
        for _ in range(100):
            # display(f'{i}')
            S_d, T_d = next(DT_loader)
            S_data, S_targets = S_d[0].to(device), S_d[1].to(device)
            T_data, T_targets = T_d[0].to(device), T_d[1].to(device)
            L_S = config.metric.loss_func(model(S_data), S_targets)
            L_T = config.metric.loss_func(model(T_data), T_targets)
            loss = (1 - alpha) * L_S + alpha * L_T
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            mean_loss.append(loss.item())
        mean_loss = np.mean(mean_loss)
        epoch_pbar.set_description(f'ML: {mean_loss:.6f}')

        stop_count += 1
        if mean_loss < lowest_loss:
            stop_count = 0
            lowest_loss = mean_loss
        if stop_count >= 5 or mean_loss < tol:
            break
    return model

@torch.no_grad()
def test_on_env(model, loader, device):
    model.eval()
    test_loss = 0
    test_acc = 0
    for data, target in loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        test_loss += config.metric.loss_func(output, target, reduction='sum').item()
        test_acc += config.metric.score_func(target, output) * len(data)
    test_loss /= len(loader.sampler)
    test_acc /= len(loader.sampler)
    return test_loss, test_acc

def save_exp(Z, filename):
    Z.to_csv(save_path / filename)

def load_exp(filename):
    Z = pd.read_csv(save_path / filename, index_col=[0, 1], header=0)
    Z.columns = np.round(Z.columns.astype(float), 2)
    return Z

In [ ]:
from tqdm.notebook import tqdm

reset_random_seed(520)

Full_D_S_loader = FastDataLoader(D_S,
                            weights=None,
                            batch_size=config.train.train_bs,
                            num_workers=config.num_workers)
Full_D_T_loader = FastDataLoader(D_T,
                            weights=None,
                            batch_size=config.train.train_bs,
                            num_workers=config.num_workers)

alpha = np.concatenate([np.arange(0, 0.29, 0.05), np.arange(0.4, 1.1, 0.2)]).round(2)
beta = np.arange(0, 0.29, 0.05).clip(0.001, 0.999).round(3)
m = 2000
exp1_Z = pd.concat([pd.DataFrame(index=beta, columns=alpha, dtype=float) for _ in range(4)],
                          keys=['loss_S', 'loss_T', 'acc_S', 'acc_T'])

never_source = True
device = config.device

dh = display(exp1_Z, display_id=True)

train_pbar = tqdm(exp1_Z.loc['loss_S'].unstack(-1).index)
for al, be in train_pbar:
    model = train_new_model(alpha=al, beta=be, device=device, m=m, source_trained=False)
    test_loss_S, test_acc_S = test_on_env(model, Full_D_S_loader, device=device)
    test_loss_T, test_acc_T = test_on_env(model, Full_D_T_loader, device=device)

    exp1_Z.loc[(slice(None), be), al] = test_loss_S, test_loss_T, test_acc_S, test_acc_T
    train_pbar.set_description(f'Source L/A: {test_loss_S:.4f}/{test_acc_S:.4f}, Target L/A: {test_loss_T:.4f}/{test_acc_T:.4f}')
    dh.update(exp1_Z)


In [5]:
%matplotlib notebook
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib

matplotlib.get_backend()
# matplotlib.use('TkAgg')

from matplotlib.backend_bases import MouseButton
import traceback
exp1_Z = load_exp('exp1.csv')
Z = exp1_Z.loc['loss_T'].clip(0, 1)
Z.columns = 1 - Z.columns
Z.index = 1 - Z.index
Z_np = Z.to_numpy()

fig, ax = plt.subplots(1, 2, subplot_kw={"projection": "3d"}, figsize=(10, 5))

ax[1].plot_surface(*np.meshgrid(Z.columns, Z.index), Z_np, vmin=Z_np.min(), cmap=cm.Blues, alpha=0.7)
ax[1].plot(Z.columns[:len(Z.index)], Z.index, Z.unstack().loc[list(zip(Z.columns[:len(Z.index)], Z.index))], zdir='z', label=r'$w_0=\lambda_0$')
ax[1].scatter(Z.idxmin(1),
           Z.index,
           Z.min(1).values,
           c='red', label=r'Loss minimum for each $\lambda_0$')

# ax.set(xticklabels=[],
#        yticklabels=[],
#        zticklabels=[])

for ax_i in ax:
    ax_i.set_xlabel(r'$w_0$', size=14)
    ax_i.set_ylabel(r'$\lambda_0$', size=14)
    ax_i.set_zlabel(r'$\epsilon_T$', size=14)
ax[1].legend(loc=1)
# ax[1].set_title("Target loss surface")

ax[0].plot_surface(*np.meshgrid(Z.columns, Z.index), Z_np, vmin=Z_np.min(), cmap=cm.Blues, alpha=1)

error_log = []

def on_move(event):
    try:
        if event.button == MouseButton.LEFT and event.inaxes in ax:
            # Get current rotation
            curr_elev, curr_azim = event.inaxes.elev, event.inaxes.azim

            # Calculate the new rotation angles based on mouse movement
            # log.append(f'{event.ydata}')
            new_elev = curr_elev - event.ydata + event.lastevent.ydata
            new_azim = curr_azim - event.xdata + event.lastevent.xdata
            # log.append(f'{new_elev}, {new_azim}')

            # Update the rotation for both axes and redraw
            for ax_i in ax:
                ax_i.view_init(elev=new_elev, azim=new_azim)
                ax_i.figure.canvas.draw_idle()
    except Exception as e:
        traceback.print_exc()
        error_log.append(f'{traceback.format_exc()}')


cid = fig.canvas.mpl_connect('motion_notify_event', on_move)
# fig.suptitle('Target loss surface [m=2000 no-S-pretrain]', fontsize=16)

# ax[0].set_zlim(Z_np.min(), 5)
plt.show()

<IPython.core.display.Javascript object>

In [8]:
exp1_Z

0.00      0.05      0.10      0.15      0.20      0.25  \
loss_S 0.001  0.006263  0.007063  0.008287  0.000638  0.005833  0.005680   
       0.050  0.022557  0.013471  0.015489  0.016648  0.011388  0.019958   
       0.100  0.036507  0.033268  0.036507  0.020938  0.017298  0.022409   
       0.150  0.032388  0.038281  0.058588  0.031888  0.028016  0.043919   
       0.200  0.042967  0.050972  0.047362  0.050550  0.057548  0.034873   
       0.250  0.080130  0.063128  0.057424  0.066384  0.070135  0.054879   
loss_T 0.001  1.283454  1.820528  1.482696  2.972042  1.720131  2.193962   
       0.050  1.314331  0.620749  0.923013  0.861736  0.716508  0.919030   
       0.100  1.361601  0.554408  0.661989  0.595763  0.664227  0.611826   
       0.150  1.210669  0.530829  0.583085  0.451819  0.499981  0.495372   
       0.200  1.288027  0.514829  0.434140  0.465689  0.476383  0.402468   
       0.250  1.582741  0.479030  0.427671  0.422874  0.442446  0.468431   
acc_S  0.001  0.997559  0.997559  0.998047  1.000000  0.998047  0.998535   
       0.050  0.995117  0.995605  0.996582  0.993652  0.996582  0.994629   
       0.100  0.988281  0.990723  0.990723  0.993164  0.993164  0.994141   
       0.150  0.990234  0.991211  0.986328  0.992188  0.989746  0.989258   
       0.200  0.985840  0.985352  0.984375  0.984375  0.985840  0.988770   
       0.250  0.980469  0.981934  0.985840  0.983887  0.982910  0.981934   
acc_T  0.001  0.711693  0.670352  0.721548  0.602692  0.637664  0.630093   
       0.050  0.699676  0.828867  0.795097  0.797380  0.839683  0.803028   
       0.100  0.691383  0.849417  0.833794  0.850138  0.845451  0.850499   
       0.150  0.699676  0.852782  0.860594  0.880663  0.878380  0.875015   
       0.200  0.684173  0.865401  0.875736  0.870328  0.877419  0.884028   
       0.250  0.673116  0.871770  0.880663  0.881985  0.887393  0.880543   

                  0.40      0.60      0.80       1.00  
loss_S 0.001  0.003854  0.011253  0.004942  16.924438  
       0.050  0.015129  0.014956  0.011657   1.622296  
       0.100  0.031447  0.026883  0.045238   1.101960  
       0.150  0.028881  0.051967  0.067644   0.713955  
       0.200  0.031355  0.071653  0.059883   0.630875  
       0.250  0.062902  0.085325  0.086635   0.697948  
loss_T 0.001  1.489847  2.928639  2.104620  20.110625  
       0.050  0.806016  0.691924  0.818460   1.234782  
       0.100  0.624794  0.665488  0.692027   0.943018  
       0.150  0.452813  0.652483  0.505445   0.661528  
       0.200  0.538914  0.565634  0.556562   0.687833  
       0.250  0.419113  0.513980  0.509094   0.612434  
acc_S  0.001  0.998535  0.997070  0.999512   0.139160  
       0.050  0.995117  0.995117  0.996094   0.592285  
       0.100  0.990723  0.990723  0.990234   0.673340  
       0.150  0.991699  0.986816  0.979492   0.802734  
       0.200  0.988281  0.979004  0.983398   0.819336  
       0.250  0.981934  0.975586  0.978027   0.802246  
acc_T  0.001  0.677322  0.610864  0.576133   0.166326  
       0.050  0.820815  0.824420  0.802908   0.654248  
       0.100  0.852662  0.844610  0.841245   0.736089  
       0.150  0.879942  0.854585  0.873573   0.808677  
       0.200  0.880663  0.872131  0.872491   0.823939  
       0.250  0.892681  0.883427  0.890638   0.841485

## Experiment 2: Source pre-trained model with $\alpha$ and $\beta$ changed.

In [ ]:
from tqdm.notebook import tqdm

reset_random_seed(520)

Full_D_S_loader = FastDataLoader(D_S,
                            weights=None,
                            batch_size=config.train.train_bs,
                            num_workers=config.num_workers)
Full_D_T_loader = FastDataLoader(D_T,
                            weights=None,
                            batch_size=config.train.train_bs,
                            num_workers=config.num_workers)

alpha = np.concatenate([np.arange(0, 0.29, 0.05), np.arange(0.4, 1.1, 0.2)]).round(2)
beta = np.arange(0, 0.29, 0.05).clip(0.001, 0.999).round(3)
m = 2000
exp2_Z = pd.concat([pd.DataFrame(index=beta, columns=alpha, dtype=float) for _ in range(4)],
                          keys=['loss_S', 'loss_T', 'acc_S', 'acc_T'])

never_source = True
device = config.device

dh = display(exp2_Z, display_id=True)

train_pbar = tqdm(exp2_Z.loc['loss_S'].unstack(-1).index)
for al, be in train_pbar:
    model = train_new_model(alpha=al, beta=be, device=device, m=m, source_trained=True)
    test_loss_S, test_acc_S = test_on_env(model, Full_D_S_loader, device=device)
    test_loss_T, test_acc_T = test_on_env(model, Full_D_T_loader, device=device)

    exp2_Z.loc[(slice(None), be), al] = test_loss_S, test_loss_T, test_acc_S, test_acc_T
    train_pbar.set_description(f'Source L/A: {test_loss_S:.4f}/{test_acc_S:.4f}, Target L/A: {test_loss_T:.4f}/{test_acc_T:.4f}')
    # display(f'alpha/beta: {al}/{be}, Source L/A: {test_loss_S:.4f}/{test_acc_S:.4f}, Target L/A: {test_loss_T:.4f}/{test_acc_T:.4f}')
    dh.update(exp2_Z)

save_exp(exp2_Z, 'exp2.csv')

In [7]:
from matplotlib.backend_bases import MouseButton
import traceback

exp2_Z = load_exp('exp2.csv')
Z = exp2_Z.loc['loss_T'].clip(0, 1)
Z.columns = 1 - Z.columns
Z.index = 1 - Z.index
Z_np = Z.to_numpy()

fig, ax = plt.subplots(1, 2, subplot_kw={"projection": "3d"}, figsize=(10, 5))

ax[1].plot_surface(*np.meshgrid(Z.columns, Z.index), Z_np, vmin=Z_np.min(), cmap=cm.Blues, alpha=0.7)
ax[1].plot(Z.columns[:len(Z.index)], Z.index, Z.unstack().loc[list(zip(Z.columns[:len(Z.index)], Z.index))], zdir='z', label=r'$w_0=\lambda_0$')
ax[1].scatter(Z.idxmin(1),
           Z.index,
           Z.min(1).values,
           c='red', label=r'Loss minimum for each $\lambda_0$')

# ax.set(xticklabels=[],
#        yticklabels=[],
#        zticklabels=[])

for ax_i in ax:
    ax_i.set_xlabel(r'$w_0$', size=14)
    ax_i.set_ylabel(r'$\lambda_0$', size=14)
    ax_i.set_zlabel(r'$\epsilon_T$', size=14)
ax[1].legend(loc=1)
# ax[1].set_title("Target loss surface")

ax[0].plot_surface(*np.meshgrid(Z.columns, Z.index), Z_np, vmin=Z_np.min(), cmap=cm.Blues, alpha=1)

error_log = []

def on_move(event):
    try:
        if event.button == MouseButton.LEFT and event.inaxes in ax:
            # Get current rotation
            curr_elev, curr_azim = event.inaxes.elev, event.inaxes.azim

            # Calculate the new rotation angles based on mouse movement
            # log.append(f'{event.ydata}')
            new_elev = curr_elev - event.ydata + event.lastevent.ydata
            new_azim = curr_azim - event.xdata + event.lastevent.xdata
            # log.append(f'{new_elev}, {new_azim}')

            # Update the rotation for both axes and redraw
            for ax_i in ax:
                ax_i.view_init(elev=new_elev, azim=new_azim)
                ax_i.figure.canvas.draw_idle()
    except Exception as e:
        traceback.print_exc()
        error_log.append(f'{traceback.format_exc()}')


cid = fig.canvas.mpl_connect('motion_notify_event', on_move)
# fig.suptitle('Target loss surface [m=2000 S-pretrain]', fontsize=16)

# ax[0].set_zlim(Z_np.min(), 5)
plt.show()

<IPython.core.display.Javascript object>

## Experiment3: No pretrained model with $m=500$ for different $\alpha$ and $\beta$

In [ ]:
from tqdm.notebook import tqdm

reset_random_seed(520)

Full_D_S_loader = FastDataLoader(D_S,
                            weights=None,
                            batch_size=config.train.train_bs,
                            num_workers=config.num_workers)
Full_D_T_loader = FastDataLoader(D_T,
                            weights=None,
                            batch_size=config.train.train_bs,
                            num_workers=config.num_workers)

alpha = np.arange(0, 1.1, 0.2).round(2)
beta = np.arange(0, 1.1, 0.2).clip(0.002, 0.998).round(3)
m = 500
exp3_Z = pd.concat([pd.DataFrame(index=beta, columns=alpha, dtype=float) for _ in range(4)],
                          keys=['loss_S', 'loss_T', 'acc_S', 'acc_T'])

never_source = True
device = config.device

dh = display(exp3_Z, display_id=True)

train_pbar = tqdm(exp3_Z.loc['loss_S'].unstack(-1).index)
for al, be in train_pbar:
    model = train_new_model(alpha=al, beta=be, device=device, m=m, source_trained=False)
    test_loss_S, test_acc_S = test_on_env(model, Full_D_S_loader, device=device)
    test_loss_T, test_acc_T = test_on_env(model, Full_D_T_loader, device=device)

    exp3_Z.loc[(slice(None), be), al] = test_loss_S, test_loss_T, test_acc_S, test_acc_T
    train_pbar.set_description(f'Source L/A: {test_loss_S:.4f}/{test_acc_S:.4f}, Target L/A: {test_loss_T:.4f}/{test_acc_T:.4f}')
    # display(f'alpha/beta: {al}/{be}, Source L/A: {test_loss_S:.4f}/{test_acc_S:.4f}, Target L/A: {test_loss_T:.4f}/{test_acc_T:.4f}')
    dh.update(exp3_Z)

save_exp(exp3_Z, 'exp3.csv')

In [9]:
from matplotlib.backend_bases import MouseButton
import traceback

exp3_Z = load_exp('exp3.csv')
Z = exp3_Z.loc['loss_T'].clip(0, 1)
Z_np = Z.to_numpy()

fig, ax = plt.subplots(1, 2, subplot_kw={"projection": "3d"}, figsize=(10, 5))

ax[1].plot_surface(*np.meshgrid(Z.columns, Z.index), Z_np, vmin=Z_np.min(), cmap=cm.Blues, alpha=0.7)
ax[1].plot(Z.columns, Z.index, Z.unstack().loc[list(zip(Z.columns, Z.index))], zdir='z', label=r'$\alpha=\beta$')
ax[1].scatter(Z.idxmin(1),
           Z.index,
           Z.min(1).values,
           c='red', label=r'Loss minimum for each $\beta$')

# ax.set(xticklabels=[],
#        yticklabels=[],
#        zticklabels=[])

for ax_i in ax:
    ax_i.set_xlabel(r'$\alpha$', size=14)
    ax_i.set_ylabel(r'$\beta$', size=14)
    ax_i.set_zlabel(r'$\epsilon_\mathcal{T}$', size=14)
ax[1].legend(loc=1)
# ax[1].set_title("Target loss surface")

ax[0].plot_surface(*np.meshgrid(Z.columns, Z.index), Z_np, vmin=Z_np.min(), cmap=cm.Blues, alpha=1)

error_log = []

def on_move(event):
    try:
        if event.button == MouseButton.LEFT and event.inaxes in ax:
            # Get current rotation
            curr_elev, curr_azim = event.inaxes.elev, event.inaxes.azim

            # Calculate the new rotation angles based on mouse movement
            # log.append(f'{event.ydata}')
            new_elev = curr_elev - event.ydata + event.lastevent.ydata
            new_azim = curr_azim - event.xdata + event.lastevent.xdata
            # log.append(f'{new_elev}, {new_azim}')

            # Update the rotation for both axes and redraw
            for ax_i in ax:
                ax_i.view_init(elev=new_elev, azim=new_azim)
                ax_i.figure.canvas.draw_idle()
    except Exception as e:
        traceback.print_exc()
        error_log.append(f'{traceback.format_exc()}')


cid = fig.canvas.mpl_connect('motion_notify_event', on_move)
fig.suptitle('Target loss surface [m=500 no-S-pretrain]', fontsize=16)

# ax[0].set_zlim(Z_np.min(), 5)
plt.show()

<IPython.core.display.Javascript object>

## Experiment4: Source-trained model with $m=500$ for different $\alpha$ and $\beta$

In [ ]:
from tqdm.notebook import tqdm

reset_random_seed(520)

Full_D_S_loader = FastDataLoader(D_S,
                            weights=None,
                            batch_size=config.train.train_bs,
                            num_workers=config.num_workers)
Full_D_T_loader = FastDataLoader(D_T,
                            weights=None,
                            batch_size=config.train.train_bs,
                            num_workers=config.num_workers)

alpha = np.arange(0, 1.1, 0.2).round(2)
beta = np.arange(0, 1.1, 0.2).clip(0.002, 0.998).round(3)
m = 500
exp4_Z = pd.concat([pd.DataFrame(index=beta, columns=alpha, dtype=float) for _ in range(4)],
                          keys=['loss_S', 'loss_T', 'acc_S', 'acc_T'])

never_source = True
device = config.device

dh = display(exp4_Z, display_id=True)

train_pbar = tqdm(exp4_Z.loc['loss_S'].unstack(-1).index)
for al, be in train_pbar:
    model = train_new_model(alpha=al, beta=be, device=device, m=m, source_trained=True)
    test_loss_S, test_acc_S = test_on_env(model, Full_D_S_loader, device=device)
    test_loss_T, test_acc_T = test_on_env(model, Full_D_T_loader, device=device)

    exp4_Z.loc[(slice(None), be), al] = test_loss_S, test_loss_T, test_acc_S, test_acc_T
    train_pbar.set_description(f'Source L/A: {test_loss_S:.4f}/{test_acc_S:.4f}, Target L/A: {test_loss_T:.4f}/{test_acc_T:.4f}')
    # display(f'alpha/beta: {al}/{be}, Source L/A: {test_loss_S:.4f}/{test_acc_S:.4f}, Target L/A: {test_loss_T:.4f}/{test_acc_T:.4f}')
    dh.update(exp4_Z)
save_exp(exp4_Z, 'exp4.csv')

In [10]:
from matplotlib.backend_bases import MouseButton
import traceback

exp4_Z = load_exp('exp4.csv')
Z = exp4_Z.loc['loss_T'].clip(0, 1)
Z_np = Z.to_numpy()

fig, ax = plt.subplots(1, 2, subplot_kw={"projection": "3d"}, figsize=(10, 5))

ax[1].plot_surface(*np.meshgrid(Z.columns, Z.index), Z_np, vmin=Z_np.min(), cmap=cm.Blues, alpha=0.7)
ax[1].plot(Z.columns, Z.index, Z.unstack().loc[list(zip(Z.columns, Z.index))], zdir='z', label=r'$\alpha=\beta$')
ax[1].scatter(Z.idxmin(1),
           Z.index,
           Z.min(1).values,
           c='red', label=r'Loss minimum for each $\beta$')

# ax.set(xticklabels=[],
#        yticklabels=[],
#        zticklabels=[])

for ax_i in ax:
    ax_i.set_xlabel(r'$\alpha$', size=14)
    ax_i.set_ylabel(r'$\beta$', size=14)
    ax_i.set_zlabel(r'$\epsilon_\mathcal{T}$', size=14)
ax[1].legend(loc=1)
# ax[1].set_title("Target loss surface")

ax[0].plot_surface(*np.meshgrid(Z.columns, Z.index), Z_np, vmin=Z_np.min(), cmap=cm.Blues, alpha=1)

error_log = []

def on_move(event):
    try:
        if event.button == MouseButton.LEFT and event.inaxes in ax:
            # Get current rotation
            curr_elev, curr_azim = event.inaxes.elev, event.inaxes.azim

            # Calculate the new rotation angles based on mouse movement
            # log.append(f'{event.ydata}')
            new_elev = curr_elev - event.ydata + event.lastevent.ydata
            new_azim = curr_azim - event.xdata + event.lastevent.xdata
            # log.append(f'{new_elev}, {new_azim}')

            # Update the rotation for both axes and redraw
            for ax_i in ax:
                ax_i.view_init(elev=new_elev, azim=new_azim)
                ax_i.figure.canvas.draw_idle()
    except Exception as e:
        traceback.print_exc()
        error_log.append(f'{traceback.format_exc()}')


cid = fig.canvas.mpl_connect('motion_notify_event', on_move)
fig.suptitle('Target loss surface', fontsize=16)

# ax[0].set_zlim(Z_np.min(), 5)
plt.show()

<IPython.core.display.Javascript object>

In [11]:
from matplotlib.backend_bases import MouseButton
import traceback

Z = exp4_Z.loc['loss_S'].clip(0, 1)
Z_np = Z.to_numpy()

fig, ax = plt.subplots(1, 2, subplot_kw={"projection": "3d"}, figsize=(10, 5))

ax[1].plot_surface(*np.meshgrid(Z.columns, Z.index), Z_np, vmin=Z_np.min(), cmap=cm.Blues, alpha=0.7)
ax[1].plot(Z.columns, Z.index, Z.unstack().loc[list(zip(Z.columns, Z.index))], zdir='z', label=r'$\alpha=\beta$')
ax[1].scatter(Z.idxmin(1),
           Z.index,
           Z.min(1).values,
           c='red', label=r'Loss minimum for each $\beta$')

# ax.set(xticklabels=[],
#        yticklabels=[],
#        zticklabels=[])

for ax_i in ax:
    ax_i.set_xlabel(r'$\alpha$', size=14)
    ax_i.set_ylabel(r'$\beta$', size=14)
    ax_i.set_zlabel(r'$\epsilon_\mathcal{S}$', size=14)
ax[1].legend(loc=1)
# ax[1].set_title("Target loss surface")

ax[0].plot_surface(*np.meshgrid(Z.columns, Z.index), Z_np, vmin=Z_np.min(), cmap=cm.Blues, alpha=1)

error_log = []

def on_move(event):
    try:
        if event.button == MouseButton.LEFT and event.inaxes in ax:
            # Get current rotation
            curr_elev, curr_azim = event.inaxes.elev, event.inaxes.azim

            # Calculate the new rotation angles based on mouse movement
            # log.append(f'{event.ydata}')
            new_elev = curr_elev - event.ydata + event.lastevent.ydata
            new_azim = curr_azim - event.xdata + event.lastevent.xdata
            # log.append(f'{new_elev}, {new_azim}')

            # Update the rotation for both axes and redraw
            for ax_i in ax:
                ax_i.view_init(elev=new_elev, azim=new_azim)
                ax_i.figure.canvas.draw_idle()
    except Exception as e:
        traceback.print_exc()
        error_log.append(f'{traceback.format_exc()}')


cid = fig.canvas.mpl_connect('motion_notify_event', on_move)
fig.suptitle('Source loss surface', fontsize=16)

plt.show()

<IPython.core.display.Javascript object>

In [13]:
from matplotlib.backend_bases import MouseButton
import traceback

Z = (2 * exp4_Z.loc['loss_S'] + exp4_Z.loc['loss_T']).clip(0, 4)
Z_np = Z.to_numpy()

fig, ax = plt.subplots(1, 2, subplot_kw={"projection": "3d"}, figsize=(10, 5))

ax[1].plot_surface(*np.meshgrid(Z.columns, Z.index), Z_np, vmin=Z_np.min(), cmap=cm.Blues, alpha=0.6)
ax[1].plot(Z.columns, Z.index, Z.unstack().loc[list(zip(Z.columns, Z.index))], zdir='z', label=r'$\alpha=\beta$')
ax[1].scatter(Z.idxmin(1),
           Z.index,
           Z.min(1).values,
           c='red', label=r'Loss minimum for each $\beta$')
ax[1].scatter(Z.min().idxmin(),
           Z.min(1).idxmin(),
           Z.min().min(),
              marker='x',
           c='green', s=60, label='Global minimum')

# ax.set(xticklabels=[],
#        yticklabels=[],
#        zticklabels=[])

for ax_i in ax:
    ax_i.set_xlabel(r'$\alpha$', size=14)
    ax_i.set_ylabel(r'$\beta$', size=14)
    ax_i.set_zlabel(r'$\epsilon_\mathcal{T} + \epsilon_\mathcal{S}$', size=14)
ax[1].legend(loc=1)
# ax[1].set_title("Target loss surface")

ax[0].plot_surface(*np.meshgrid(Z.columns, Z.index), Z_np, vmin=Z_np.min(), cmap=cm.Blues, alpha=1)

error_log = []

def on_move(event):
    try:
        if event.button == MouseButton.LEFT and event.inaxes in ax:
            # Get current rotation
            curr_elev, curr_azim = event.inaxes.elev, event.inaxes.azim

            # Calculate the new rotation angles based on mouse movement
            # log.append(f'{event.ydata}')
            new_elev = curr_elev - event.ydata + event.lastevent.ydata
            new_azim = curr_azim - event.xdata + event.lastevent.xdata
            # log.append(f'{new_elev}, {new_azim}')

            # Update the rotation for both axes and redraw
            for ax_i in ax:
                ax_i.view_init(elev=new_elev, azim=new_azim)
                ax_i.figure.canvas.draw_idle()
    except Exception as e:
        traceback.print_exc()
        error_log.append(f'{traceback.format_exc()}')


cid = fig.canvas.mpl_connect('motion_notify_event', on_move)
fig.suptitle('Source loss + Target loss surface', fontsize=16)

plt.show()

<IPython.core.display.Javascript object>

## Fix $m_T$, increase $m_S$, how $\epsilon_T$ changes

In [16]:
t = np.arange(1, 101)
mt = t * 2 #+ 10
d = 50
delta = 1e-2

ms = t * 30 #+ 50
m = ms + mt
alpha = np.arange(0, 1, 1e-1)
t, alpha = np.meshgrid(t, alpha)
beta = mt / m

# * np.log(2 * m + 2)
Z = 4 * np.sqrt((alpha **2) / beta + ((1-alpha) ** 2) / (1 - beta)) * np.sqrt((2 * d  + 2 * np.log(8/delta))/ m) + 2 * (1 - alpha) * 0.715
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

ax.plot_surface(t, alpha, Z.clip(2, 5), vmin=0, cmap=cm.Blues, alpha=1)
ax.set_xlabel(r'$t$', size=14)
ax.set_ylabel(r'$\alpha$', size=14)
ax.set_zlabel(r'$\epsilon_\mathcal{T}$', size=14)

fig.suptitle('Target loss', fontsize=16)

plt.show()

<IPython.core.display.Javascript object>

In [17]:
beta

array([0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625,
       0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625,
       0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625,
       0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625,
       0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625,
       0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625,
       0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625,
       0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625,
       0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625,
       0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625,
       0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625,
       0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625,
       0.0625, 0.0625, 0.0625, 0.0625])